<a href="https://colab.research.google.com/github/benthecoder/code-notebooks/blob/main/eth_prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-05-20,"$2,439.64","$2,993.15","$2,170.23","$2,784.29","$2,784.29",67610826680
2021-05-21,"$2,772.34","$2,938.21","$2,113.35","$2,430.62","$2,430.62",53774070802
2021-05-22,"$2,436.01","$2,483.98","$2,168.12","$2,295.71","$2,295.71",42089937660
2021-05-23,"$2,298.37","$2,384.41","$1,737.47","$2,109.58","$2,109.58",56005721977
2021-05-24,"$2,145.73","$2,544.05","$2,091.07","$2,532.89","$2,532.89",53217079296


In [ ]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1967 entries, 2016-01-01 to 2021-05-24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1967 non-null   float64
 1   High       1967 non-null   float64
 2   Low        1967 non-null   float64
 3   Close      1967 non-null   float64
 4   Adj Close  1967 non-null   float64
 5   Volume     1967 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 107.6 KB


In [ ]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [ ]:
df.tail()

,ds,y
1962,2021-05-20,"$2,439.64"
1963,2021-05-21,"$2,772.34"
1964,2021-05-22,"$2,436.01"
1965,2021-05-23,"$2,298.37"
1966,2021-05-24,"$2,145.73"


In [ ]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [ ]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2327,2022-05-20
2328,2022-05-21
2329,2022-05-22
2330,2022-05-23
2331,2022-05-24


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2327,2022-05-20,"$5,852.34","$3,563.80","$7,976.08"
2328,2022-05-21,"$5,660.44","$3,394.47","$7,724.64"
2329,2022-05-22,"$5,457.99","$3,287.26","$7,428.27"
2330,2022-05-23,"$5,231.01","$3,158.21","$7,164.88"
2331,2022-05-24,"$5,054.01","$3,019.28","$6,942.42"


In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

2525.5788173913843

In [ ]:
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)